In [1]:
# Now import the objects we need
from dotenv import load_dotenv
# Other needed packages to import
import os
import requests
import json

# Weather Forecast API

- Information about the Weather API
    - Request URL: http://api.weatherapi.com/v1/forecast.json
    - Parameters:
        - Key: "0b84826a7cc24575b16232249242804"
        - q (query parameter): Ex: q = Paris
        - days: days = 3

In [22]:
# Load environment variables from .env file
load_dotenv()
# Now you can access the environment variable
weather_api_key = os.getenv('WEATHER_API_KEY')
# You can print the key to make sure it is there, but I get nervous when I see a key printed somewhere.... Someone could steal it!
print(weather_api_key)

0b84826a7cc24575b16232249242804


In [23]:
# Function to get weather forecast API (GET())
def get_weather_forecast(city, num_days):
    # Base URL of the weather API
    base_url = "http://api.weatherapi.com/v1/forecast.json"

    # Parameters to be sent to the API
    params = {
        'key': weather_api_key,
        'q': city,
        'days': num_days
    }

    # Sending the GET request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parsing the JSON response
        data = response.json()
        return data
    else:
        # Print an error message if something went wrong
        return "Failed to retrieve data: " + response.text

In [38]:
# Input travel destination
city = 'San Luis Obispo'
num_days = 3

# Use the get_weather_forecast function to call API for the city inputted above
forecast_data = get_weather_forecast(city, num_days)
forecast_data

{'location': {'name': 'San Luis Obispo',
  'region': 'California',
  'country': 'United States of America',
  'lat': 35.28,
  'lon': -120.66,
  'tz_id': 'America/Los_Angeles',
  'localtime_epoch': 1714427491,
  'localtime': '2024-04-29 14:51'},
 'current': {'last_updated_epoch': 1714427100,
  'last_updated': '2024-04-29 14:45',
  'temp_c': 20.0,
  'temp_f': 68.0,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 23.0,
  'wind_kph': 37.1,
  'wind_degree': 310,
  'wind_dir': 'NW',
  'pressure_mb': 1015.0,
  'pressure_in': 29.97,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 49,
  'cloud': 0,
  'feelslike_c': 20.0,
  'feelslike_f': 68.0,
  'vis_km': 16.0,
  'vis_miles': 9.0,
  'uv': 6.0,
  'gust_mph': 27.5,
  'gust_kph': 44.3},
 'forecast': {'forecastday': [{'date': '2024-04-29',
    'date_epoch': 1714348800,
    'day': {'maxtemp_c': 20.9,
     'maxtemp_f': 69.6,
     'mintemp_c': 8.0,
     

In [39]:
# Reduce .JSON Forecast Data to reduce data fed to ChatGPT
# Simplify the forecast data
simplified_forecast = []
for day in forecast_data["forecast"]["forecastday"]:
    simplified_info = {
        "date": day["date"],
        "max_temp": day["day"]["maxtemp_c"],
        "min_temp": day["day"]["mintemp_c"],
        "condition": day["day"]["condition"]["text"],
        "chance_of_rain": day["day"]["daily_chance_of_rain"]
    }
    simplified_forecast.append(simplified_info)

simplified_forecast

[{'date': '2024-04-29',
  'max_temp': 20.9,
  'min_temp': 8.0,
  'condition': 'Sunny',
  'chance_of_rain': 0},
 {'date': '2024-04-30',
  'max_temp': 21.8,
  'min_temp': 8.3,
  'condition': 'Sunny',
  'chance_of_rain': 0},
 {'date': '2024-05-01',
  'max_temp': 22.9,
  'min_temp': 9.2,
  'condition': 'Sunny',
  'chance_of_rain': 0}]

# ChatGPT API Call

In [40]:
# Load environment variables from .env file
load_dotenv()
# Now you can access the environment variable
openai_api_key = os.getenv('OPENAI_API_KEY')
# You can print the key to make sure it is there, but I get nervous when I see a key printed somewhere.... Someone could steal it!
print(openai_api_key)

sk-proj-SS1ePlDHkJ5EsqrkDqTwT3BlbkFJXqZ01MUt2Dg8mh34sPUO


In [42]:
# Define the API endpoint
url = "https://api.openai.com/v1/chat/completions"

# Set the request headers
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai_api_key}"
}

In [43]:
# Define the data
#data = {
#    "model": "gpt-4", # models: https://platform.openai.com/docs/models/overview
#    'temperature': 1,
#    "max_tokens": 500,
#    "messages": [
#       {
#            "role": "user","content": "Create a list of fun activities I should do while visiting San Luis Obispo, California, USA. Explain why you recommend each activity and the best weather for the activity."
#        }
 #   ]
#}

In [51]:
# Define the data
data = {
    "model": "gpt-4", # models: https://platform.openai.com/docs/models/overview
    'temperature': 1,
    "max_tokens": 500,
    "messages": [
        {
            "role": "user",
            "content": f"The following .JSON contains weather forecast information for the city of {city} for a certain number of days. I want you to create a list of recommended activities based on the city and the weather forecast. Please list the activities, which day you recommend the activities, explain why you recommend each activity, and why it will be good according to the forecast. Here is the forecast information: <{simplified_forecast}>"
        }
    ]
}

In [52]:
# Make the API POST request.
response = requests.post(url, headers=headers, data=json.dumps(data))
# Handle the response
if response.status_code == 200: # All went well
    print('The API call was successful.')
    # Store the response content into a variable
    generated_text = response.json()['choices'][0]['message']['content']
else:
    print(f"Error: {response.status_code} - {response.text}")
#
# When you execute this cell, you are paying to inference the model. It will deduct money from your account.
# Therefore, I usually do anything in this cell other than store the response.

The API call was successful.


In [53]:
# Look at the resposne
print(generated_text)

1. Activity: Bike riding at Bob Jones City to Sea Bike Trail
   Day: April 29th, 2024
   Reason: The weather is sunny, with a high of 20.9 degrees Celsius, ideal for outdoor activities. The Bob Jones Trail is a popular trail in the city and a scenic spot for biking.
   Compatibility with forecast: With zero percent chance of rain, it would be a perfect day to engage in outdoor physical activities.

2. Activity: Visit The San Luis Obispo Museum of Art
   Day: April 30th, 2024
   Reason: After physical activity on the previous day, take a slower paced day and visit the local art museum in the city. The weather will be sunny, and though the museum is indoors, it will be enjoyable to explore the surroundings as well. The max temp is 21.8, another lovely day in San Luis Obispo.
   Compatibility with forecast: The sunny weather makes for a nice walk to the museum. Also, there's no chance of rain, which is ideal for moving around the city.

3. Activity: Hiking at Bishop Peak
   Day: May 1st, 